In [2]:
import joblib
import numpy as np

X_train = joblib.load('./trials/features/hfcc150_pitch_range/X_train.joblib')
X_test = joblib.load('./trials/features/hfcc150_pitch_range/X_test.joblib')

y_train = joblib.load(r'trials\features\48k_mfcc_extra_hfcc_extra\y_train.joblib')
y_test = joblib.load(r'trials\features\48k_mfcc_extra_hfcc_extra\y_test.joblib')

X_train2= joblib.load(r'trials\features\48k_mfcc_extra_hfcc_extra\X_train.joblib')
X_test2= joblib.load(r'trials\features\48k_mfcc_extra_hfcc_extra\X_test.joblib')

X_train_combined = np.concatenate((X_train2[:,:150], X_train2[:,7*75:9*75], X_train[:,-1:]), axis=1)
X_test_combined = np.concatenate((X_test2[:,:150], X_test2[:,7*75:9*75], X_test[:,-1:]), axis=1)

x_gender_train = np.concatenate((X_train2[:,:150], X_train2[:,7*75:9*75], X_train[:,-1:]), axis=1)
y_gender_train = y_train % 2

x_gender_test = np.concatenate((X_test2[:,:150], X_test2[:,7*75:9*75], X_test[:,-1:]), axis=1)
y_gender_test = y_test % 2

x_age_train = np.concatenate((X_train2[:,:150], X_train2[:,7*75:9*75], X_train[:,-1:]), axis=1)
y_age_train = y_train // 2

x_age_test = np.concatenate((X_test2[:,:150], X_test2[:,7*75:9*75], X_test[:,-1:]), axis=1)
y_age_test = y_test // 2

female_train = y_train % 2 == 1
x_female_train_all = np.concatenate((X_train2[:,:150], X_train2[:,525:675],X_train2[:,750:900],X_train2[:,975:1050]), axis=1)
x_female_train = x_female_train_all[female_train]
y_female_train = y_train[female_train]

female_test = y_test % 2 == 1
x_female_test_all = np.concatenate((X_test2[:,:150], X_test2[:,525:675],X_test2[:,750:900],X_test2[:,975:1050]), axis=1)
x_female_test = x_female_test_all[female_test]
y_female_test = y_test[female_test]

male_train = y_train % 2 == 0
x_male_train_all = np.concatenate((X_train2[:,:150], X_train2[:,7*75:9*75]), axis=1)
x_male_train = x_male_train_all[male_train]
y_male_train = y_train[male_train] // 2

male_test = y_test % 2 == 0
x_male_test_all = np.concatenate((X_test2[:,:150], X_test2[:,7*75:9*75]), axis=1)
x_male_test = x_male_test_all[male_test]
y_male_test = y_test[male_test] // 2

In [3]:
from sklearn.utils import resample

def oversample_minority(x, y):
  unique, counts = np.unique(y, return_counts=True)
  oversample_value = unique[np.argmin(counts)]
  true_indices = np.where(y == oversample_value)[0]
  false_indices = np.where(y != oversample_value)[0]

  true_oversampled = resample(
    true_indices,
    replace=True,
    n_samples=len(false_indices),
    random_state=42
  )

  balanced_indices = np.concatenate([false_indices, true_oversampled])
  np.random.shuffle(balanced_indices)

  x_balanced = x[balanced_indices]
  y_balanced = y[balanced_indices]

  return x_balanced, y_balanced

def undersample_majority(x, y):
  unique, counts = np.unique(y, return_counts=True)
  undersample_value = unique[np.argmax(counts)]
  true_indices = np.where(y == undersample_value)[0]
  false_indices = np.where(y != undersample_value)[0]
  
  true_undersampled = resample(
    true_indices,
    replace=False,
    n_samples=len(false_indices),
    random_state=42
  )

  balanced_indices = np.concatenate([false_indices, true_undersampled])
  np.random.shuffle(balanced_indices)

  x_balanced = x[balanced_indices]
  y_balanced = y[balanced_indices]

  return x_balanced, y_balanced

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC

y_class_0 = y_train == 0

skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
train_idx, val_idx = next(skf.split(X_train_combined, y_class_0))

X_train_fold, X_val_fold = X_train_combined[train_idx], X_train_combined[val_idx]
y_train_fold, y_val_fold = y_class_0[train_idx], y_class_0[val_idx]

clf = SVC(kernel='rbf', C=1000, gamma=0.0001, random_state=42, class_weight='balanced')
clf.fit(X_train_fold, y_train_fold)

y_val_pred_0 = clf.predict(X_val_fold)
y_val_pred_proba_0 = clf.predict_proba(X_val_fold)
print('Accuracy: %.3f' % (accuracy_score(y_val_fold, y_val_pred_0)))
print(classification_report(y_val_fold, y_val_pred_0))

In [110]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

y_class_1 = y_train == 1

skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
train_idx, val_idx = next(skf.split(X_train_combined, y_class_1))

X_train_fold, X_val_fold = X_train_combined[train_idx], X_train_combined[val_idx]
y_train_fold, y_val_fold = y_class_1[train_idx], y_class_1[val_idx]

knn = KNeighborsClassifier(n_neighbors=3, weights='distance', n_jobs=-1)
knn.fit(X_train_fold, y_train_fold)

y_val_pred_1 = knn.predict(X_val_fold)
y_val_pred_proba_1 = knn.predict_proba(X_val_fold)
print('Accuracy: %.3f' % (accuracy_score(y_val_fold, y_val_pred_1)))
print(classification_report(y_val_fold, y_val_pred_1))

Accuracy: 0.983
              precision    recall  f1-score   support

       False       0.99      0.99      0.99      7610
        True       0.96      0.89      0.92      1030

    accuracy                           0.98      8640
   macro avg       0.97      0.94      0.96      8640
weighted avg       0.98      0.98      0.98      8640



In [111]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

y_class_2 = y_train == 2

skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
train_idx, val_idx = next(skf.split(X_train_combined, y_class_2))

X_train_fold, X_val_fold = X_train_combined[train_idx], X_train_combined[val_idx]
y_train_fold, y_val_fold = y_class_2[train_idx], y_class_2[val_idx]

knn = KNeighborsClassifier(n_neighbors=3, weights='distance', n_jobs=-1)
knn.fit(X_train_fold, y_train_fold)

y_val_pred_2 = knn.predict(X_val_fold)
y_val_pred_proba_2 = knn.predict_proba(X_val_fold)
print('Accuracy: %.3f' % (accuracy_score(y_val_fold, y_val_pred_2)))
print(classification_report(y_val_fold, y_val_pred_2))

Accuracy: 0.978
              precision    recall  f1-score   support

       False       0.99      0.99      0.99      7718
        True       0.90      0.90      0.90       922

    accuracy                           0.98      8640
   macro avg       0.94      0.94      0.94      8640
weighted avg       0.98      0.98      0.98      8640



In [112]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

y_class_3 = y_train == 3

skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
train_idx, val_idx = next(skf.split(X_train_combined, y_class_3))

X_train_fold, X_val_fold = X_train_combined[train_idx], X_train_combined[val_idx]
y_train_fold, y_val_fold = y_class_3[train_idx], y_class_3[val_idx]
X_train_balanced, y_train_balanced = X_train_fold, y_train_fold

knn = KNeighborsClassifier(n_neighbors=3, weights='distance', n_jobs=-1)
knn.fit(X_train_balanced, y_train_balanced)

y_val_pred_3 = knn.predict(X_val_fold)
y_val_pred_proba_3 = knn.predict_proba(X_val_fold)
print('Accuracy: %.3f' % (accuracy_score(y_val_fold, y_val_pred_3)))
print(classification_report(y_val_fold, y_val_pred_3))

Accuracy: 0.991
              precision    recall  f1-score   support

       False       0.99      0.99      0.99      7858
        True       0.95      0.95      0.95       782

    accuracy                           0.99      8640
   macro avg       0.97      0.97      0.97      8640
weighted avg       0.99      0.99      0.99      8640



In [115]:
# Stack the predictions for each class
y_val_preds = np.vstack([y_val_pred_0, y_val_pred_1, y_val_pred_2, y_val_pred_3])
y_val_preds_proba = np.vstack([y_val_pred_proba_0[:, 1].T, y_val_pred_proba_1[:, 1].T, y_val_pred_proba_2[:, 1].T, y_val_pred_proba_3[:, 1].T])

# Count how many classifiers predict True for each example
num_true_preds = np.sum(y_val_preds, axis=0)

# Count how many examples have more than one classifier predicting True
num_multiple_true = np.sum(num_true_preds > 1)

print(f"Number of examples with multiple classifiers predicting True: {num_multiple_true}/{len(y_val_preds[0])}")
print(f"Number of examples with no classifier predicting True: {sum(num_true_preds == 0)}/{len(y_val_preds[0])}")

overall_pred = np.argmax(y_val_preds, axis=0)
print('Accuracy: %.3f' % (accuracy_score(y_val_fold, overall_pred)))

overall_pred_proba = np.argmax(y_val_preds_proba, axis=0)
print('Accuracy: %.3f' % (accuracy_score(y_val_fold, overall_pred_proba)))

Number of examples with multiple classifiers predicting True: 1742/8640
Number of examples with no classifier predicting True: 1984/8640
Accuracy: 0.852
Accuracy: 0.822


In [117]:
from IPython.display import Image, display

display(Image(url="https://media.tenor.com/ekebXEH4uBEAAAAS/monkey-laught.gif"))